In [2]:
try:
    import keras
    from keras.datasets import mnist
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Flatten, LSTM
    from keras.layers import Conv1D, MaxPool1D
    from keras import backend as K
except:
    raise ImportError("For this example you need to install keras.")

try:
    import torchvision
    import torchvision.transforms as transforms
except:
    raise ImportError("For this example you need to install pytorch-vision.")

import scipy.io
import numpy as np
import tensorflow as tf
import sklearn as sk
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
from tensorflow.keras.callbacks import  EarlyStopping
import random

gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    
  try:
      
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
      
  except RuntimeError as e:
      
    print(e)
    
class MyModelCheckpoint(tf.keras.callbacks.ModelCheckpoint):

    def __init__(self, *args, **kwargs):
        super(MyModelCheckpoint, self).__init__(*args, **kwargs)

    # redefine the save so it only activates after 100 epochs
    def on_epoch_end(self, epoch, logs=None):
        if epoch >= 1: super(MyModelCheckpoint, self).on_epoch_end(epoch, logs)

# 기존 train 데이터 불러오기
# [11, 505000, 2]
train_sub_data = scipy.io.loadmat('../datas/160hz/train_5day_160hz.mat', squeeze_me=True)['data']

# 기존 test 데이터 불러오기
# [11, 101402, 2]
test_sub_data = scipy.io.loadmat('../datas/160hz/test_6day_160hz(2).mat', squeeze_me=True)['data']

# 데이터 크기를 480로 맞춰줌
# 5554560/480 = 115,72
data_size = 480
train_sub_size = 1052 # 1명당 3초 데이터 1052개
test_sub_size = 211

# # 데이터를 480(= 160*3) 크기로 사용할 수 있도록 그 배수로 전체 데이터 잘라줌
data_cut_size = 504960 # 480*1052 = 504960
test_cut_size = 101280 # 480*211 = 101280

# 트레인 sub 수
train_sub_cnt = train_sub_data.shape[0]
# test 데이터 sub 수
test_sub_cnt = test_sub_data.shape[0]

# sub_cnt = train_sub_cnt + public_sub_cnt

train_sub_cut = train_sub_data[:, 0:data_cut_size, :]
test_sub_cut = test_sub_data[:, 0:test_cut_size, :]

print(train_sub_cut.shape)
print(test_sub_cut.shape)

# 데이터를 스케일링 하기 위해 2D로 reshape
train_sub_2D = train_sub_cut.reshape(-1, 1)
test_2D = test_sub_cut.reshape(-1, 1)

# 데이터를 StandardScaler로 스케일링
# 데이터들 모두 같은 방식으로 스케일링함
SDscaler = StandardScaler()
SDscaler.fit(train_sub_2D)
train_scaled = SDscaler.transform(train_sub_2D)
test_scaled = SDscaler.transform(test_2D)

train_data = train_scaled.reshape(train_sub_cnt * train_sub_size, data_size, 2)
test_data = test_scaled.reshape(test_sub_cnt * test_sub_size, data_size, 2)

print(train_data.shape)
print(test_data.shape)

#test data를 sub:other=1:1로 만들기 위해서 각 sub 추출
test_data_each = []
for i in range(test_sub_cnt):
    test_data_each.insert(i, test_data[i*test_sub_size:(i+1)*test_sub_size, :, :])
# print(test_data_each)

#train data를 sub:other=1:1로 만들기 위해서 각 sub 추출
train_data_each = []
for i in range(train_sub_cnt):
    train_data_each.insert(i, train_data[i*train_sub_size:(i+1)*train_sub_size, :, :])
    
# sub index
i = 5
# 1 to ratio
ratio = 3

train_data_n = train_data_each[i]
test_data_n = test_data_each[i]

cnt = 0
for j in range(train_sub_cnt):
    if j != i and cnt < 4:
        cnt = cnt + 1
        train_data_n = np.append(train_data_n, np.array(random.sample(list(train_data_each[j]), 315)), axis = 0)
    elif j != i and cnt >= 4:
        train_data_n = np.append(train_data_n, np.array(random.sample(list(train_data_each[j]), 316)), axis = 0)

cnt = 0
for j in range(test_sub_cnt):
    if j != i and cnt < 7:
        cnt = cnt + 1
        test_data_n = np.append(test_data_n, np.array(random.sample(list(test_data_each[j]), 63)), axis = 0)
    elif j != i and cnt >= 7:
        test_data_n = np.append(test_data_n, np.array(random.sample(list(test_data_each[j]), 64)), axis = 0)

train_label = np.zeros(train_sub_size*(ratio+1))
test_label = np.zeros(test_sub_size*(ratio+1))

for j in range(len(train_label)):
    if (j < train_sub_size):
        train_label[j] = 1

for j in range(len(test_label)):
    if (j < test_sub_size):
        test_label[j] = 1
        
# train 데이터를 train과 validation으로 분배
train_data_shuffled, train_label_shuffled = sk.utils.shuffle(train_data_n, train_label, random_state = 0)

val_data_set = train_data_shuffled[:train_sub_size]
train_data_set = train_data_shuffled[train_sub_size:]

val_label_set = train_label_shuffled[:train_sub_size]
train_label_set = train_label_shuffled[train_sub_size:]

import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

from hpbandster.core.worker import Worker

import logging
logging.basicConfig(level=logging.DEBUG)


# 작업자 클래스
class KerasWorker(Worker):
    def __init__(self, **kwargs):
            super().__init__(**kwargs)

            self.batch_size = 64

            img_rows = 480
            img_cols = 2
            self.num_classes = 1
            
            x_train, y_train = train_data_set, train_label_set
            x_validation, y_validation = val_data_set, val_label_set
            x_test, y_test   = test_data_n, test_label

            if K.image_data_format() == 'channels_first':
                    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols)
                    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols)
                    self.input_shape = (img_rows, img_cols)
            else:
                    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols)
                    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols)
                    self.input_shape = (img_rows, img_cols)

            self.x_train, self.y_train = x_train, y_train
            self.x_validation, self.y_validation = x_validation, y_validation
            self.x_test, self.y_test   = x_test, y_test
            
            self.input_shape = (img_rows, img_cols)


    def compute(self, config, budget, working_directory, *args, **kwargs):
            """
            Simple example for a compute function using a feed forward network.
            It is trained on the MNIST dataset.
            The input parameter "config" (dictionary) contains the sampled configurations passed by the bohb optimizer
            """

            model = Sequential()
            
            # Cnn 층
            model.add(Conv1D(config['num_filters_1'], kernel_size=3, strides = 1, padding = 'same',
                            activation='relu', input_shape=self.input_shape))
            
            model.add(MaxPool1D(pool_size=3))
            
            if config['num_conv_layers'] > 1:
                model.add(Conv1D(config['num_filters_2'], kernel_size=3, strides = 1, padding = 'same',
                            activation='relu', input_shape=self.input_shape))
                model.add(MaxPool1D(pool_size=3))
                
            if config['num_conv_layers'] > 2:
                model.add(Conv1D(config['num_filters_3'], kernel_size=3, strides = 1, padding = 'same',
                            activation='relu', input_shape=self.input_shape))
                model.add(MaxPool1D(pool_size=3))
                
            
            if config['num_conv_layers'] > 3:
                model.add(Conv1D(config['num_filters_4'], kernel_size=3, strides = 1, padding = 'same',
                            activation='relu', input_shape=self.input_shape))
                model.add(MaxPool1D(pool_size=3))
            
                
            if config['num_conv_layers'] > 4:
                model.add(Conv1D(config['num_filters_5'], kernel_size=3, strides = 1, padding = 'same',
                            activation='relu', input_shape=self.input_shape))
                model.add(MaxPool1D(pool_size=3))

            model.add(Dropout(config['dropout_rate']))
            
            # LSTM 층
            model.add(LSTM(config['lstm_units']))
            
            # Dense 층
            model.add(Dense(config['num_fc_units_1'], activation='relu'))
            
            if config['num_fc_layers'] > 1:
                model.add(Dense(config['num_fc_units_2'], activation='relu'))
            
            if config['num_fc_layers'] > 2:
                model.add(Dense(config['num_fc_units_3'], activation='relu'))
            
            model.add(Dense(self.num_classes, activation='sigmoid'))
            
            if config['optimizer'] == 'Adam':
                    optimizer = keras.optimizers.Adam(lr=config['lr'])
            else:
                    optimizer = keras.optimizers.SGD(lr=config['lr'], momentum=config['sgd_momentum'])

            model.compile(loss=keras.losses.categorical_crossentropy,
                                      optimizer=optimizer,
                                      metrics=['accuracy'])

            if config['optimizer'] == 'Adam':
                    optimizer = keras.optimizers.Adam(lr=config['lr'])
            else:
                    optimizer = keras.optimizers.SGD(lr=config['lr'], momentum=config['sgd_momentum'])

            model.compile(loss=keras.losses.binary_crossentropy,
                                      optimizer=optimizer,
                                      metrics=['accuracy'])

            model.fit(self.x_train, self.y_train,
                              batch_size=self.batch_size,
                              epochs=int(budget),
                              verbose=0,
                              validation_data=(self.x_validation, self.y_validation)
                     )

            train_score = model.evaluate(self.x_train, self.y_train, verbose=0)
            val_score = model.evaluate(self.x_validation, self.y_validation, verbose=0)
            test_score = model.evaluate(self.x_test, self.y_test, verbose=0)

            #import IPython; IPython.embed()
            return ({
                    'loss': 1-val_score[1],
                    'info': {       'test accuracy': test_score[1],
                                            'train accuracy': train_score[1],
                                            'validation accuracy': val_score[1],
                                            'number of parameters': model.count_params(),
                                    }

            })


    @staticmethod
    def get_configspace():
            """
            It builds the configuration space with the needed hyperparameters.
            It is easily possible to implement different types of hyperparameters.
            Beside float-hyperparameters on a log scale, it is also able to handle categorical input parameter.
            :return: ConfigurationsSpace-Object
            """
            cs = CS.ConfigurationSpace()

            lr = CSH.UniformFloatHyperparameter('lr', lower=1e-6, upper=1e-1, default_value='1e-2', log=True)

            # For demonstration purposes, we add different optimizers as categorical hyperparameters.
            # To show how to use conditional hyperparameters with ConfigSpace, we'll add the optimizers 'Adam' and 'SGD'.
            # SGD has a different parameter 'momentum'.
            optimizer = CSH.CategoricalHyperparameter('optimizer', ['Adam', 'SGD'])

            sgd_momentum = CSH.UniformFloatHyperparameter('sgd_momentum', lower=0.0, upper=0.99, default_value=0.9, log=False)

            cs.add_hyperparameters([lr, optimizer, sgd_momentum])



            num_conv_layers =  CSH.UniformIntegerHyperparameter('num_conv_layers', lower=1, upper=5, default_value=3)

            num_filters_1 = CSH.UniformIntegerHyperparameter('num_filters_1', lower=4, upper=256, default_value=16, log=True)
            num_filters_2 = CSH.UniformIntegerHyperparameter('num_filters_2', lower=4, upper=256, default_value=32, log=True)
            num_filters_3 = CSH.UniformIntegerHyperparameter('num_filters_3', lower=4, upper=256, default_value=64, log=True)
            num_filters_4 = CSH.UniformIntegerHyperparameter('num_filters_4', lower=4, upper=256, default_value=128, log=True)
            num_filters_5 = CSH.UniformIntegerHyperparameter('num_filters_5', lower=4, upper=256, default_value=256, log=True)

            cs.add_hyperparameters([num_conv_layers, num_filters_1, num_filters_2, num_filters_3, num_filters_4, num_filters_5])

            dropout_rate = CSH.UniformFloatHyperparameter('dropout_rate', lower=0.0, upper=0.9, default_value=0.5, log=False)
            
            lstm_units = CSH.UniformIntegerHyperparameter('lstm_units', lower=8, upper=256, default_value=64, log=False)
            
            num_fc_layers =  CSH.UniformIntegerHyperparameter('num_fc_layers', lower=1, upper=3, default_value=2) 
            
            num_fc_units_1 = CSH.UniformIntegerHyperparameter('num_fc_units_1', lower=8, upper=256, default_value=128, log=True)
            num_fc_units_2 = CSH.UniformIntegerHyperparameter('num_fc_units_2', lower=8, upper=256, default_value=64, log=True)
            num_fc_units_3 = CSH.UniformIntegerHyperparameter('num_fc_units_3', lower=8, upper=256, default_value=32, log=True)

            cs.add_hyperparameters([dropout_rate, lstm_units, num_fc_layers, num_fc_units_1, num_fc_units_2, num_fc_units_3])


            # The hyperparameter sgd_momentum will be used,if the configuration
            # contains 'SGD' as optimizer.
            cond = CS.EqualsCondition(sgd_momentum, optimizer, 'SGD')
            cs.add_condition(cond)

            # You can also use inequality conditions:
            cond = CS.GreaterThanCondition(num_filters_2, num_conv_layers, 1)
            cs.add_condition(cond)

            cond = CS.GreaterThanCondition(num_filters_3, num_conv_layers, 2)
            cs.add_condition(cond)

            cond = CS.GreaterThanCondition(num_filters_4, num_conv_layers, 3)
            cs.add_condition(cond)

            cond = CS.GreaterThanCondition(num_filters_5, num_conv_layers, 4)
            cs.add_condition(cond)
            
            cond = CS.GreaterThanCondition(num_fc_units_2, num_fc_layers, 1)
            cs.add_condition(cond)

            cond = CS.GreaterThanCondition(num_fc_units_3, num_fc_layers, 2)
            cs.add_condition(cond)

            return cs




if __name__ == "__main__":
    
    # 작업자 시작
    worker = KerasWorker(run_id='0')
    
    # 옵티마이저 실행
    cs = worker.get_configspace()

    # 결과 분석
    config = cs.sample_configuration().get_dictionary()
    print(config)
    res = worker.compute(config=config, budget=2, working_directory='../BOHB_LAB_5CNN')
    print(res)

(11, 504960, 2)
(11, 101280, 2)
(11572, 480, 2)
(2321, 480, 2)
{'dropout_rate': 0.8350894246115568, 'lr': 0.04435978356692676, 'lstm_units': 154, 'num_conv_layers': 3, 'num_fc_units_1': 75, 'num_fc_units_2': 53, 'num_filters_1': 4, 'optimizer': 'Adam', 'num_filters_2': 4, 'num_filters_3': 37}
{'loss': 0.26330798864364624, 'info': {'test accuracy': 0.75, 'train accuracy': 0.7544360160827637, 'validation accuracy': 0.7366920113563538, 'number of parameters': 134540}}
